<a href="https://colab.research.google.com/github/barbedorafael/Python_workshop_HGE/blob/main/using_python_for_web_scrapting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install selenium

In [5]:
pip install bs4

In [6]:
'''
web scrapting
INMET data

'''

import os
import time

# Importar as funções que iremos utilizar do Selenium
from selenium import webdriver

# Importar Selenium
from bs4 import BeautifulSoup 

In [ ]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [49]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
#chrome_options.add_argument('--disable-dev-shm-usage')
browser = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

#Atribuir ao Selenium o site que será acessado
browser.get('https://tempo.inmet.gov.br/TabelaEstacoes/')

#Recomendado para evitar ban do servidor
time.sleep(3)

#Selecionar em "Produto" a opção "Tabela de Dados das Estações" 
browser.find_element_by_xpath("//select/option[@value='TabelaEstacoes']").click()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  


In [50]:
VALOR_ESTACAO='A801' #Porto Alegre - RS

DATA_INIT='01/01/2022'
DATA_END='02/02/2022'


#Selecionamos a opção de estação automática
browser.find_element_by_xpath("//select/option[@value='T']").click()

#Selecionamos qual estação estamos interessados
browser.find_element_by_xpath("//select/option[@value='"+VALOR_ESTACAO+"']").click()

#Primeiro limpados o formulário e então preenchemos com a data inicial que desejamos.

browser.find_element_by_id("datepicker_EstacoesTabela_Inicio").clear()
browser.find_element_by_id("datepicker_EstacoesTabela_Inicio").send_keys(DATA_INIT)

#O mesmo para a data final
browser.find_element_by_id("datepicker_EstacoesTabela_Fim").clear()
browser.find_element_by_id("datepicker_EstacoesTabela_Fim").send_keys(DATA_END)

#Por fim, clicamos em "gerar tabela"
browser.find_element_by_id("EstacoesTabela").click()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:

In [51]:
#Pedimos para o Selenium aguardar por alguns segundos até que a tabela seja gerada pelo site
WebDriverWait(browser, 10).until(EC.visibility_of_element_located((By.ID, "tabela")))

#atribuimos a estrutura atual do site para uma variável para que o BeautifulSoup possa fazer sua mágica!
page_source = browser.page_source

#"Limpamos" a estrutura do site com o BeautifulSoup 
soup = BeautifulSoup(page_source, 'lxml')

#Pede para retornar a tabela existente na tag "table"
table = soup.find('table')

In [55]:
import pandas as pd

#Converter a tabela html em Dataframe. 
#Definimos como separador decimal ',' e milhar '.'
df = pd.read_html(str(table),decimal=',', thousands='.')[0]
df=pd.DataFrame(df.to_records())

#Se preferir, também há como renomear o cabeçalho da planilha.

New_Names=['index','Date','Time','Tinst','Tmax','Tmin','URinst','URmax','URmin','PtOrvalhoinst','PtOrvalhomax','PtOrvalhmin',
          'Pinst','Pmax','Pmin','Vveloc','Vdir','Vraj','Rad','Precipt'
          ]

for n in  range(0,len(df.keys())):
    
    df=df.rename(columns={df.keys()[n]: New_Names[n]})


In [59]:
df

,index,Date,Time,Tinst,Tmax,Tmin,URinst,URmax,URmin,PtOrvalhoinst,PtOrvalhomax,PtOrvalhmin,Pinst,Pmax,Pmin,Vveloc,Vdir,Vraj,Rad,Precipt,datetime
0,0,01/01/2022,0,25.0,26.4,25.0,72.0,72.0,65.0,19.7,19.7,19.3,1000.3,1000.3,999.4,2.1,126.0,5.4,NaN,0.0,2022-01-01
1,1,01/01/2022,100,24.1,25.0,24.1,77.0,77.0,72.0,19.8,19.8,19.6,1000.6,1000.6,1000.2,1.3,95.0,4.3,NaN,0.0,2022-01-01
2,2,01/01/2022,200,23.6,24.1,23.6,80.0,80.0,77.0,19.9,19.9,19.8,1000.5,1000.8,1000.5,1.0,76.0,3.4,NaN,0.0,2022-01-01
3,3,01/01/2022,300,23.3,23.8,23.3,79.0,80.0,78.0,19.6,19.9,19.5,1000.0,1000.5,1000.0,1.0,104.0,2.6,NaN,0.0,2022-01-01
4,4,01/01/2022,400,23.0,23.6,22.5,84.0,85.0,78.0,20.1,20.2,19.4,999.7,1000.0,999.6,1.4,132.0,3.4,NaN,0.0,2022-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
787,787,02/02/2022,1900,30.4,33.8,30.4,57.0,58.0,45.0,21.0,21.7,19.7,1005.3,1005.7,1005.3,2.3,98.0,6.2,1992.29,0.0,2022-02-02
788,788,02/02/2022,2000,30.6,31.7,30.4,58.0,58.0,52.0,21.5,21.8,20.1,1005.2,1005.5,1005.2,3.2,104.0,7.2,2020.42,0.0,2022-02-02
789,789,02/02/2022,2100,28.7,30.6,28.7,62.0,62.0,58.0,20.7,21.4,20.0,1005.1,1005.2,1005.1,2.9,106.0,7.9,1332.62,0.0,2022-02-02
790,790,02/02/2022,2200,26.9,28.7,26.9,72.0,72.0,62.0,21.5,21.5,20.7,1005.1,1005.2,1005.0,2.3,100.0,8.9,289.57,0.0,2022-02-02
